# SARIMAとは？
## ARモデル(自己回帰モデル)
「過去の自分のデータ」を説明変数とする。
Φの値によっては定常モデルになる
$$
    y_t = c + \phi_1 y_{t-1} + \phi_2 y_{t-2} +･･･ + \phi_p y_{t-p} + \epsilon_t
$$
## MAモデル(移動平均モデル)
たとえば「今日の売上が予想より多ければ明日の売上は少なくなる」というような現象をモデリングできる。
定常なモデル  


$$
    y_t = c + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} +･･･ + \theta_q \epsilon_{t-q} + \epsilon_t
$$

## ARMAモデル(ARモデル+MAモデル)
定常性を有する時系列データに対して有効
$$
    y = c + \sum_{i=1}^{p}\phi_iy_{t-i} + \sum_{j=1}^q\theta_j\epsilon_{j-q} + \epsilon_t\\
$$

## ARIMAモデル
ARMAを定常性がないモデルにまで拡張したモデル。差分をとることで定常モデルとして扱えるようデータを加工する。
$$\Delta y = y_t-y_{t-1}$$
一階差分で定常性が見れないときは二階差分、三階差分...d階差分と拡張する。
$$\Delta^2 y = \Delta y_t-\Delta y_{t-1} \quad \Delta^3 y = \Delta^2 y_t-\Delta^2 y_{t-1}$$

## SARIMAモデル (Seasonal ARIMA model)
ARIMAモデルに周期成分を取り入れたモデル。  
時系列方向の説明にARIMA(p,d,q) モデルを使うだけでなく、周期方向の説明にもARIMA(P,D,Q)モデルを利用する。  
SARIMAモデルでは合計7個の次数がある。 時系列方向のARIMA(p,d,q)に加え季節差分方向のARIMA(P,D,Q)と周期 s がある。  
これをSARIMA(p,d,q)(P,D,Q)\[s\]と表記する。  
次数の組み合わせ爆発の問題を回避するため、周期 s は作図や自己相関関数をもとに決め打ちし、季節差分の P,D,Qは低く抑えることがよく行われる。

参考:  
https://ai-trend.jp/basic-study/time-series-analysis/sarima_model/  
http://by-oneself.com/r_ar_ma/


# statsmodels: 線形回帰、時系列問題を解くライブラリ  
参考1: [statsmodelsによる線形回帰 入門](https://qiita.com/innovation1005/items/b712ce54a7a697a9bf03)  
参考2: [Statsmodelsによる時系列分析入門](https://qiita.com/innovation1005/items/6c5263d79ccc67263b2c)  

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

from scipy import stats

In [ ]:
#基本ライブラリ
import os
from datetime import datetime
import time
from itertools import product
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

# グラフ
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

info = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/asset_details.csv")
ctrain = pd.read_csv("/kaggle/input/g-research-crypto-forecasting/train.csv")

In [ ]:
# Data Selection
def crypto_sub(asset_id ,data= ctrain ):
    df = data[data["Asset_ID"]==asset_id].reset_index(drop = True)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.set_index('timestamp')
    return df

#Subgroup Bitcoin
btc1=crypto_sub(asset_id=1)

#Subgroup Eth
eth1=crypto_sub(asset_id=6)

#Subgroup Ada
ada1=crypto_sub(asset_id=3)

In [ ]:
# Function to Subset the Price variables
def mini_data(data):
    df=data[["Close","Low","High","Open"]]
    return df

btc1_mini=mini_data(btc1)
eth1_mini=mini_data(eth1)
ada1_mini=mini_data(ada1)
btc1.head()

# 各通貨の周期性をチェック
どの通貨においても周期性は確認できなかった=ARIMAモデルが適している

In [ ]:
# Function to Plot
plt.rcParams["figure.figsize"]=(15,7)

def season_day(data):
    df=data.resample("D").mean()
    seasonal_decompose(df.Close).plot()
    return plt.show()

def season_month(data):
    df=data.resample("M").mean()
    seasonal_decompose(df.Close).plot()
    return plt.show()

In [ ]:
season_day(data=btc1_mini)
season_month(data=btc1_mini)

In [ ]:
season_day(data=eth1_mini)
season_month(data=eth1_mini)

In [ ]:
season_day(data=ada1_mini)
season_month(data=ada1_mini)

# Box-Cox変換
変数を正規分布に近しい形に変換する
$$
    y^{(\lambda)} = (x^\lambda - 1)/\lambda \quad (\lambda \neq 0) \\
    y^{(\lambda)} = ln(x) \quad (\lambda = 0)
$$


In [ ]:
btc_month=btc1_mini.resample("M").mean()
btc_month["close_box"], lmbda=stats.boxcox(btc_month.Close)

In [ ]:
# p,qの値を0~3で設定し、一番良いmodelを探す
# dは1で固定
qs = range(0, 3)
ps = range(0, 3)
d=1
parameters_list = list(product(ps, qs))

results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')
for param in parameters_list:
    # close_boxを説明変数にしてmodel学習
    try:
        model = SARIMAX(
            btc_month.close_box,
            order=(param[0], d, param[1]),
            # seasonal_order = ()   # 周期性は見られなかったので入力しない=ARIMAになる
        ).fit(disp=-1)
    except ValueError:
        print('bad parameter combination:', param)
        continue
    aic = model.aic
    # 最も良いmodelとaicの値をbest_modelとbest_aicに格納する
    # (値が小さいほど良い)
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

In [ ]:
# modelを結果が良い順に並べる
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())

In [ ]:
# 一番良い結果
print(best_model.summary())

In [ ]:
best_param

In [ ]:
best_model.plot_diagnostics(figsize=(15, 12))
plt.show()
# https://blog.amedama.jp/entry/sm-decompose-series

In [ ]:
# 逆Box-Cox変換
def invboxcox(y,lmbda):
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda*y+1)/lmbda))

In [ ]:
# Prediction
btc_month_pred = btc_month[['Close']]
date_list = [datetime(2021, 6, 20), datetime(2021, 7, 20), datetime(2021, 8, 21), datetime(2021, 9, 21)]
future = pd.DataFrame(index=date_list, columns= btc_month.columns)
btc_month_pred = pd.concat([btc_month_pred, future])
btc_month_pred['forecast'] = invboxcox(best_model.predict(start=datetime(2018, 1, 31), end=datetime(2021, 5, 31)), lmbda)
plt.figure(figsize=(18,10))
btc_month_pred.Close.plot()
btc_month_pred.forecast.plot(color='r', ls='--', label='Predicted Close')
plt.legend()
plt.title('Bitcoin monthly forecast')
plt.ylabel('USD')
plt.show()

In [ ]:
# # 提出
# env = gresearch_crypto.make_env()
# iter_test = env.iter_test()

In [ ]:
# df_test_entire = []

# for i, (df_test, sample_prediction_df) in enumerate(iter_test):
#     # Asset_IDでモデルを分割しているため、1データ毎で予測
#     for j, test_row in df_test.iterrows():
#         # test_rowのCloseにBox-Cox変換を掛ける
#         test_row["close_box"], lmbda=stats.boxcox(test_row["Close"])
        
#         # close_boxに対してmodelを作成する
#         model = SARIMAX(
#             test_row["close_box"],
#             order=(best_param[0], d, best_param[1]),
#         ).fit(disp=-1)
#         model.predict(test_row)
#         test_row[]
    
    
#         test_row
#         env.predict([])

# import gresearch_crypto
# env = gresearch_crypto.make_env()   # initialize the environment
# iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
# for (test_df, sample_prediction_df) in iter_test:
#     sample_prediction_df['Target'] = 0  # make your predictions here
#     env.predict(sample_prediction_df)   # register your predictions